In [ ]:
import pandas as pd
import re 
import os
from langchain_groq import ChatGroq
from langchain.chains import LLMChain
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate
from langchain.memory import ConversationBufferMemory

import warnings
warnings.filterwarnings("ignore")

In [2]:
news = pd.read_csv("NewsData/USDT_News_Data.csv").drop("Unnamed: 0", axis = 1)
stocks = pd.read_csv("StocksData/USDT_Stocks_Scrapped.csv").drop("Unnamed: 0", axis = 1)

In [3]:
news

,Crypto,Title,Date
0,Tether cypto,Ukraine to Regulate Bitcoin Businesses Under E...,2014-02-17
1,Tether cypto,Dorian S. Nakamoto Denies He's Bitcoin Founder,2014-03-07
2,Tether cypto,"Swedish Kickboxer Wins Bitcoin Fight Prize, Ma...",2014-04-07
3,Tether cypto,Dorian Nakamoto Thanks Bitcoin Community for 4...,2014-04-22
4,Tether cypto,6 New Bitcoin ATMs And Where To Find Them,2014-05-22
...,...,...,...
3838,Tether cypto,"XRP Becomes Third Largest Crypto, Surpassing S...",2024-12-02
3839,Tether cypto,XRP Surges 28% To Become World’s Third-Largest...,2024-12-02
3840,Tether cypto,Crypto Investors Fear Market Volatility as Tet...,2024-12-03
3841,Tether cypto,Tether Price (USDT INR),2024-12-04


In [4]:
stocks

,Date,Open,High,Low,Close,Adj Close,Volume
0,"Nov 2, 2024",0.999528,1.000078,0.999407,0.999641,0.999641,"68,466,413,568"
1,"Oct 31, 2024",0.999596,1.000572,0.998211,0.998623,0.998623,"67,445,977,914"
2,"Oct 30, 2024",0.999712,1.000788,0.998488,0.999632,0.999632,"69,142,436,437"
3,"Oct 29, 2024",0.999333,1.000320,0.998754,0.999729,0.999729,"87,586,242,107"
4,"Oct 28, 2024",0.998694,1.000287,0.998287,0.999305,0.999305,"66,855,500,410"
...,...,...,...,...,...,...,...
2545,"Nov 13, 2017",1.004480,1.029290,0.975103,1.009350,1.009350,"767,884,032"
2546,"Nov 12, 2017",1.006020,1.105910,0.967601,1.012470,1.012470,"1,466,060,032"
2547,"Nov 11, 2017",1.005980,1.026210,0.995799,1.008990,1.008990,"746,227,968"
2548,"Nov 10, 2017",1.006500,1.024230,0.995486,1.006010,1.006010,"756,446,016"


In [5]:
stocks['Date'] = pd.to_datetime(stocks['Date']).dt.strftime('%Y-%m-%d')

In [6]:
stocks

,Date,Open,High,Low,Close,Adj Close,Volume
0,2024-11-02,0.999528,1.000078,0.999407,0.999641,0.999641,"68,466,413,568"
1,2024-10-31,0.999596,1.000572,0.998211,0.998623,0.998623,"67,445,977,914"
2,2024-10-30,0.999712,1.000788,0.998488,0.999632,0.999632,"69,142,436,437"
3,2024-10-29,0.999333,1.000320,0.998754,0.999729,0.999729,"87,586,242,107"
4,2024-10-28,0.998694,1.000287,0.998287,0.999305,0.999305,"66,855,500,410"
...,...,...,...,...,...,...,...
2545,2017-11-13,1.004480,1.029290,0.975103,1.009350,1.009350,"767,884,032"
2546,2017-11-12,1.006020,1.105910,0.967601,1.012470,1.012470,"1,466,060,032"
2547,2017-11-11,1.005980,1.026210,0.995799,1.008990,1.008990,"746,227,968"
2548,2017-11-10,1.006500,1.024230,0.995486,1.006010,1.006010,"756,446,016"


In [7]:
df = pd.merge(news, stocks, on = 'Date', how = 'right').drop('Crypto', axis = 1)
df

,Title,Date,Open,High,Low,Close,Adj Close,Volume
0,Tether Brings Dirham-Pegged Stablecoin to TON ...,2024-11-02,0.999528,1.000078,0.999407,0.999641,0.999641,"68,466,413,568"
1,Tether to Launch Dirham-Pegged Stablecoin on t...,2024-11-02,0.999528,1.000078,0.999407,0.999641,0.999641,"68,466,413,568"
2,Tether Introduces Dirham-Pegged Stablecoin on ...,2024-11-02,0.999528,1.000078,0.999407,0.999641,0.999641,"68,466,413,568"
3,Tether increases Bitcoin and gold reserves in ...,2024-10-31,0.999596,1.000572,0.998211,0.998623,0.998623,"67,445,977,914"
4,Tether hits $7.7 billion in profit YTD as rese...,2024-10-31,0.999596,1.000572,0.998211,0.998623,0.998623,"67,445,977,914"
...,...,...,...,...,...,...,...,...
4515,CME Group CEO Terry Duffy: 'Bitcoin Futures Wi...,2017-11-13,1.004480,1.029290,0.975103,1.009350,1.009350,"767,884,032"
4516,NaN,2017-11-12,1.006020,1.105910,0.967601,1.012470,1.012470,"1,466,060,032"
4517,NaN,2017-11-11,1.005980,1.026210,0.995799,1.008990,1.008990,"746,227,968"
4518,NaN,2017-11-10,1.006500,1.024230,0.995486,1.006010,1.006010,"756,446,016"


In [8]:
df['Date'].nunique()

2550

In [9]:
df['Title'].isnull().sum()

893

In [10]:
df[df['Title'].isnull()]

,Title,Date,Open,High,Low,Close,Adj Close,Volume
74,NaN,2024-10-19,0.999910,1.000206,0.999238,0.999949,0.999949,"32,490,132,300"
75,NaN,2024-10-18,0.999686,1.000595,0.999141,0.999908,0.999908,"59,487,210,585"
82,NaN,2024-10-15,0.999548,1.001132,0.999203,0.999808,0.999808,"83,963,242,850"
86,NaN,2024-10-13,0.999755,1.000836,0.999315,0.999557,0.999557,"36,960,774,632"
87,NaN,2024-10-12,0.999742,1.001143,0.999421,0.999757,0.999757,"35,944,351,076"
...,...,...,...,...,...,...,...,...
4514,NaN,2017-11-14,1.005240,1.013430,0.996898,1.006830,1.006830,"429,857,984"
4516,NaN,2017-11-12,1.006020,1.105910,0.967601,1.012470,1.012470,"1,466,060,032"
4517,NaN,2017-11-11,1.005980,1.026210,0.995799,1.008990,1.008990,"746,227,968"
4518,NaN,2017-11-10,1.006500,1.024230,0.995486,1.006010,1.006010,"756,446,016"


In [ ]:
os.environ["GROQ_API_KEY"] = "Enter API Key Here"
os.environ["OPENAI_API_KEY"] = "Enter API Key Here"

In [ ]:
llm = ChatGroq(model = "llama-3.3-70b-versatile")

# Define the memory object
memory = ConversationBufferMemory()

# Define the system prompt
system_message = (
    f"""
    You are a machine learning bot with expertise in financial news, stock market trends, political science, cryptocurrencies, and international relations that impact stock prices.
    Analyze the following news title and identify its sentiment.
    Provide the result in the specified format, which includes:
    Class: Sentiment category (positive, negative, or neutral).
    Score: Sentiment confidence score as a float (0 to 1).
    Polarity: Sentiment polarity ranging from -1 (negative) to 1 (positive).
    Subjectivity: Subjectivity score between 0 (objective) and 1 (highly subjective).

    Input and Output constraints:
    Input: title: string
    Output: A JSON object with the following structure:
    'class': <sentiment>, 'score': <float>, 'polarity': <float>, 'subjectivity': <float>

    Examples:
    Example Input:
    NaN

    Example Output:
    'class': 'neutral', 'score': 0.0000, 'polarity': 0.0000, 'subjectivity': 0.0000

    Example Input:
    "Stock prices surge as investors regain confidence in tech sector."

    Example Output:
    'class': 'positive', 'score': 0.945, 'polarity': 0.8, 'subjectivity': 0.7

    Example Input:
    "Market crashes amid fears of global recession."

    Example Output:
    'class': 'negative', 'score': 0.988, 'polarity': -0.9, 'subjectivity': 0.8

    Example Input:
    "The Federal Reserve announced its decision to keep interest rates unchanged."

    Example Output:
    'class': 'neutral', 'score': 0.998, 'polarity': 0.0, 'subjectivity': 0.2

    Notes:
    Always provide output strictly in the JSON format specified above.
    Do not include any explanations, comments, or text outside of the JSON object in your response.
    """
)

# Define prompt templates
system_prompt_template = SystemMessagePromptTemplate.from_template(system_message)
human_prompt_template = HumanMessagePromptTemplate.from_template("{input}")

chat_prompt = ChatPromptTemplate.from_messages([system_prompt_template, human_prompt_template])


# Create the chain
chain = LLMChain(
    llm=llm,
    prompt=chat_prompt,
    memory=memory
)

def get_persuasion(text) -> list:
    if text is None:
        return 0
        
    response = chain.run(input=text)

    return response

In [13]:
t1 = df['Title'][21]
t1

"Inside Tether's Vaults: 82,000 Bitcoin And 48 Tons Of Gold Safeguard USDT"

In [14]:
print(get_persuasion(t1))

{'class': 'positive', 'score': 0.92, 'polarity': 0.7, 'subjectivity': 0.65}


In [15]:
import numpy as np
# Corrected implementation
lens = [len(t) for t in df['Title'] if pd.notna(t)]
np.mean(lens)

69.1532947339399

In [16]:
tdf = df.sample(n=10)
from tqdm import tqdm

# Enable tqdm for pandas apply
tqdm.pandas()

# Apply the function with progress tracking
tdf['sentiment_score'] = tdf['Title'].progress_apply(get_persuasion)

# tdf['persuasion_score'] = tdf['Title'].apply(get_persuasion)

100%|██████████| 10/10 [00:03<00:00,  2.60it/s]


In [17]:
tdf

,Title,Date,Open,High,Low,Close,Adj Close,Volume,sentiment_score
1060,China: Court Rejects Another Crypto-Related Lo...,2023-10-13,0.999646,1.000466,0.999381,0.999960,0.999960,"21,158,055,012","{'class': 'negative', 'score': 0.853, 'polarit..."
18,The WSJ heavily accuses Tether and halts the c...,2024-10-28,0.998694,1.000287,0.998287,0.999305,0.999305,"66,855,500,410","{'class': 'negative', 'score': 0.983, 'polarit..."
188,Tether: a program to educate on the use cases ...,2024-09-10,1.000152,1.001288,0.999647,1.000017,1.000017,"44,958,459,857","{'class': 'positive', 'score': 0.92, 'polarity..."
3245,Would You Like to Learn How to Play Ultimate T...,2020-03-28,1.014251,1.022956,0.994298,0.999704,0.999704,"44,315,404,128","{'class': 'neutral', 'score': 0.993, 'polarity..."
1023,NaN,2023-11-03,1.000266,1.001012,0.999968,1.000650,1.000650,"30,750,864,935","{'class': 'neutral', 'score': 0.99998021125793..."
1240,"Binance, Tether and Justin Sun let slip the do...",2023-06-22,1.000182,1.000807,0.999151,1.000184,1.000184,"31,071,097,282","{'class': 'negative', 'score': 0.892, 'polarit..."
860,"Tether prints $10B in 3 months, nears exclusiv...",2024-01-19,0.999408,1.000000,0.998467,0.999186,0.999186,"46,650,934,337","{'class': 'positive', 'score': 0.92, 'polarity..."
3952,Should Universities Create a Courses Around Bl...,2018-12-26,1.021943,1.032330,1.011707,1.019550,1.019550,"4,461,185,016","{'class': 'neutral', 'score': 0.992, 'polarity..."
4234,Third-party report says all Tether (USDT) full...,2018-06-20,1.001710,1.010210,0.994434,0.998829,0.998829,"2,215,259,904","{'class': 'positive', 'score': 0.942, 'polarit..."
3716,NaN,2019-06-07,1.004479,1.011645,1.001242,1.006650,1.006650,"18,921,026,163","{'class': 'neutral', 'score': 0.99998021125793..."


In [24]:
import openai

In [25]:
openai.api_key = os.getenv("OPENAI_API_KEY")

def get_persuasion_gpt(text):
    if text is np.nan:
        return {'class': 'neutral', 'score': 0.0000, 'polarity': 0.0000, 'subjectivity': 0.0000}
    try:
        # Replace "gpt-4o-mini" with a valid model name (e.g., "gpt-4" or "gpt-3.5-turbo")
        completion = openai.ChatCompletion.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": text}
            ]
        )
        # Extract the response content
        return completion['choices'][0]['message']['content']
    except Exception as e:
        # Log and return the error for debugging
        print(f"Error processing text: {text}\n{e}")
        return 0.0


In [26]:
tdf = df.sample(n=10)
from tqdm import tqdm

# Enable tqdm for pandas apply
tqdm.pandas()

# Apply the function with progress tracking
tdf['persuasion_score'] = tdf['Title'].progress_apply(get_persuasion_gpt)

# tdf['persuasion_score'] = tdf['Title'].apply(get_persuasion)

100%|██████████| 10/10 [00:11<00:00,  1.18s/it]


In [27]:
tdf

,Title,Date,Open,High,Low,Close,Adj Close,Volume,persuasion_score
3664,Binance Launches Platform '2.0' as Margin Trad...,2019-07-11,1.000290,1.009433,0.994273,0.996514,0.996514,"30,184,888,505","{\n ""class"": ""positive"",\n ""score"": 0.91..."
3184,Coronavirus & Altcoins: Why are Some Crypto As...,2020-05-01,1.003174,1.012035,0.998792,1.008137,1.008137,"56,216,767,268","{\n ""class"": ""neutral"",\n ""score"": 0.845..."
4288,Circle Becomes 'Bitcoin Unicorn' with Bitmain-...,2018-05-15,1.000540,1.013020,0.993853,1.001140,1.001140,"3,315,440,128","{\n ""class"": ""positive"",\n ""score"": 0.92..."
714,Coinbase Revealed To Be Backing BlackRock’s ‘$...,2024-03-22,0.999800,1.001175,0.999114,1.000266,1.000266,"81,269,693,027","{\n ""class"": ""positive"",\n ""score"": 0.87..."
4127,P2P Exchange Options Increasing for Crypto Tra...,2018-08-26,1.003210,1.007950,0.997750,0.999227,0.999227,"1,946,530,000","{\n ""class"": ""positive"",\n ""score"": 0.87..."
3033,Tether Could Get More Use Cases,2020-07-24,0.998255,1.003818,0.997001,0.998993,0.998993,"22,338,273,101","{\n ""class"": ""positive"",\n ""score"": 0.86..."
1580,Tether Stablecoins Face Renewed Scrutiny Post-FTX,2022-12-01,1.000029,1.000094,0.999956,1.000044,1.000044,"29,913,541,569","{\n ""class"": ""negative"",\n ""score"": 0.87..."
302,Tether Eyes Doubling Workforce Amid Recession ...,2024-08-09,1.000450,1.001723,0.999590,1.000578,1.000578,"55,251,256,632","{\n ""class"": ""neutral"",\n ""score"": 0.845..."
882,Crypto News Updates: Tether Emerges as Hotbed ...,2024-01-15,0.999492,1.000350,0.999076,0.999723,0.999723,"41,325,317,168","{\n ""class"": ""negative"",\n ""score"": 0.87..."
2774,Pavel Durov Reveals Telegram Plans for 2021: M...,2020-12-23,0.999687,1.000226,0.999074,0.999426,0.999426,"85,019,037,325","{\n ""class"": ""positive"",\n ""score"": 0.92..."


In [ ]:
df

,Title,Date,Open,High,Low,Close,Adj Close,Volume
0,NaN,2024-11-02,"69,476.78","69,855.11","69,476.78","69,551.83","69,551.83","44,786,638,848"
1,MicroStrategy Announces $42 Billion Bitcoin In...,2024-10-31,"72,335.05","72,662.31","69,590.50","70,215.19","70,215.19","40,627,912,076"
2,BlackRock CEO Issues Stark Fed Warning Amid $3...,2024-10-31,"72,335.05","72,662.31","69,590.50","70,215.19","70,215.19","40,627,912,076"
3,MicroStrategy Announces Third Quarter 2024 Fin...,2024-10-30,"72,715.37","72,905.30","71,411.73","72,339.54","72,339.54","40,646,637,831"
4,"Bitcoin (BTC/USD) Takes a Pause, $75k Still On...",2024-10-30,"72,715.37","72,905.30","71,411.73","72,339.54","72,339.54","40,646,637,831"
...,...,...,...,...,...,...,...,...
11374,Bitcoin API Developer Gem Raises $2 Million in...,2014-09-17,465.86,468.17,452.42,457.33,457.33,"21,056,800"
11375,Medical Marijuana Dispensary Becomes Canada’s ...,2014-09-17,465.86,468.17,452.42,457.33,457.33,"21,056,800"
11376,Louisiana State Republican Party Now Accepts B...,2014-09-17,465.86,468.17,452.42,457.33,457.33,"21,056,800"
11377,CoinJar pioneers Australia’s first bitcoin EFT...,2014-09-17,465.86,468.17,452.42,457.33,457.33,"21,056,800"


In [ ]:
df.to_csv("persuasion_gpt-3.csv")

In [29]:
train = pd.read_csv("NewsData/cryptonews.csv")
train

,date,sentiment,source,subject,text,title,url
0,2023-12-19 06:40:41,"{'class': 'negative', 'polarity': -0.1, 'subje...",CryptoNews,altcoin,Grayscale CEO Michael Sonnenshein believes the...,Grayscale CEO Calls for Simultaneous Approval ...,https://cryptonews.comhttps://cryptonews.com/n...
1,2023-12-19 06:03:24,"{'class': 'neutral', 'polarity': 0.0, 'subject...",CryptoNews,blockchain,"In an exclusive interview with CryptoNews, Man...",Indian Government is Actively Collaborating Wi...,https://cryptonews.comhttps://cryptonews.com/n...
2,2023-12-19 05:55:14,"{'class': 'positive', 'polarity': 0.05, 'subje...",CryptoNews,blockchain,According to the Federal Court ruling on Decem...,Judge Approves Settlement: Binance to Pay $1.5...,https://cryptonews.comhttps://cryptonews.com/n...
3,2023-12-19 05:35:26,"{'class': 'positive', 'polarity': 0.5, 'subjec...",CoinTelegraph,blockchain,Some suggest EVM inscriptions are the latest w...,Why a gold rush for inscriptions has broken ha...,https://cointelegraph.com/news/inscriptions-ev...
4,2023-12-19 05:31:08,"{'class': 'neutral', 'polarity': 0.0, 'subject...",CoinTelegraph,ethereum,A decision by bloXroute Labs to start censorin...,‘Concerning precedent’ — bloXroute Labs' MEV r...,https://cointelegraph.com/news/concerning-prec...
...,...,...,...,...,...,...,...
31032,2021-10-27 15:17:00,"{'class': 'neutral', 'polarity': 0.0, 'subject...",CryptoNews,defi,Cream Finance (CREAM) suffered another flash l...,Cream Finance Suffers Another Exploit as Attac...,https://cryptonews.com/news/cream-finance-suff...
31033,2021-10-19 13:39:00,"{'class': 'positive', 'polarity': 0.1, 'subjec...",CryptoNews,blockchain,Banque de France disclosed the results of its ...,French Central Bank's Blockchain Bond Trial Br...,https://cryptonews.com/news/french-central-ban...
31034,2021-10-18 13:58:00,"{'class': 'positive', 'polarity': 0.14, 'subje...",CryptoNews,blockchain,Advancing its project to become \x9caÂ\xa0meta...,"Facebook To Add 10,000 Jobs In EU For Metavers...",https://cryptonews.com/news/facebook-to-add-10...
31035,2021-10-15 00:00:00,"{'class': 'neutral', 'polarity': 0.0, 'subject...",CryptoNews,blockchain,Chinese companies are still topping the blockc...,Tech Crackdown Hasn't Halted Chinese Firms' Bl...,https://cryptonews.com/news/tech-crackdown-has...


In [33]:
temp = train.sample(frac=0.1)
temp
temp['Sentiment_LLM'] = temp['text'].progress_apply(get_persuasion_gpt)

 88%|████████▊ | 2731/3104 [48:14<04:10,  1.49it/s]   

Error processing text: Among many critical revelations about Alameda and FTX’s relationship, Caroline Ellison confirmed that Alameda always had access to customers’ funds at FTX.
HTTP code 502 from API (<html>
<head><title>502 Bad Gateway</title></head>
<body>
<center><h1>502 Bad Gateway</h1></center>
<hr><center>cloudflare</center>
</body>
</html>
)


100%|██████████| 3104/3104 [54:14<00:00,  1.05s/it]


In [64]:
import re
import pandas as pd

# Sample DataFrame (replace with your actual DataFrame)
temp = pd.read_csv("baseline_sentiment_analysis_llm.csv")
temp

# Regex pattern to match valid JSON object(s)
import re

# Regex pattern to match JSON-like structure
regex_pattern = r'\{[^{}]*\}'

# Apply regex to extract JSON-like content and handle None results
temp['Sentiment_LLM_Json'] = temp['Sentiment_LLM'].apply(
    lambda x: re.search(regex_pattern, str(x)).group(0) if isinstance(x, str) and re.search(regex_pattern, str(x)) else ''
)

# Replace newline characters in the 'Sentiment_LLM_Json' column
temp['Sentiment_LLM_Json'] = temp['Sentiment_LLM_Json'].str.replace('\n', '', regex=False)




In [65]:
temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3104 entries, 0 to 3103
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          3104 non-null   int64 
 1   date                3104 non-null   object
 2   sentiment           3104 non-null   object
 3   source              3104 non-null   object
 4   subject             3104 non-null   object
 5   text                3104 non-null   object
 6   title               3104 non-null   object
 7   url                 3104 non-null   object
 8   Sentiment_LLM       3104 non-null   object
 9   true_sentiment      3104 non-null   object
 10  Sentiment_LLM_Json  3104 non-null   object
dtypes: int64(1), object(10)
memory usage: 266.9+ KB


In [44]:
temp.to_csv("baseline_comapre_sentiment_llm.csv")

In [66]:
temp

,Unnamed: 0,date,sentiment,source,subject,text,title,url,Sentiment_LLM,true_sentiment,Sentiment_LLM_Json
0,12221,2023-04-12 10:06:00,"{'class': 'negative', 'polarity': -0.05, 'subj...",CryptoNews,altcoin,The price of Shiba Inu has dropped by 2.5% in ...,Shiba Inu Price Prediction as SHIB Rises 14% F...,https://cryptonews.com/news/shiba-inu-price-pr...,"{\n ""class"": ""negative"",\n ""score"": 0.87...","{'class': 'negative', 'polarity': -0.05, 'subj...","{ ""class"": ""negative"", ""score"": 0.872, ..."
1,23762,2022-06-28 17:30:15,"{'class': 'positive', 'polarity': 0.05, 'subje...",CoinTelegraph,blockchain,The TON blockchain has always been Proof-of-St...,‘Unique phenomenon’: All 5B toncoins mined on ...,https://cointelegraph.com/news/unique-phenomen...,"{\n ""class"": ""neutral"",\n ""score"": 0.975...","{'class': 'positive', 'polarity': 0.05, 'subje...","{ ""class"": ""neutral"", ""score"": 0.975, ..."
2,19554,2022-10-15 09:00:00,"{'class': 'positive', 'polarity': 0.13, 'subje...",CryptoNews,altcoin,XRP price is in an enviably bullish position a...,"XRP Price Prediction - If This Chart is Right,...",https://cryptonews.com/news/xrp-price-predicti...,"{\n ""class"": ""positive"",\n ""score"": 0.92...","{'class': 'positive', 'polarity': 0.13, 'subje...","{ ""class"": ""positive"", ""score"": 0.925, ..."
3,561,2023-12-07 17:45:00,"{'class': 'neutral', 'polarity': 0.0, 'subject...",CryptoPotato,bitcoin,"According to S3 Partners, short sellers have i...",BTC Short Sellers Burn Through Billions as Bit...,https://cryptopotato.com/btc-short-sellers-bur...,"{\n ""class"": ""negative"",\n ""score"": 0.94...","{'class': 'neutral', 'polarity': 0.0, 'subject...","{ ""class"": ""negative"", ""score"": 0.947, ..."
4,10313,2023-05-25 17:45:00,"{'class': 'positive', 'polarity': 0.05, 'subje...",CryptoNews,altcoin,Sensational new artificial intelligence (AI)-p...,AiDoge.com's Presale Nears $14 Million Raised ...,https://cryptonews.com/news/aidogecoms-presale...,"{\n ""class"": ""positive"",\n ""score"": 0.92...","{'class': 'positive', 'polarity': 0.05, 'subje...","{ ""class"": ""positive"", ""score"": 0.925, ..."
...,...,...,...,...,...,...,...,...,...,...,...
3099,30703,2021-12-8 16:55:00,"{'class': 'neutral', 'polarity': 0.0, 'subject...",CryptoNews,ethereum,"Get your daily, bite-sized digest of cryptoass...","USD 3B Crypto Trading Floor, Bitmart Resumes E...",https://cryptonews.com/news/usd-3b-crypto-trad...,"{\n ""class"": ""neutral"",\n ""score"": 0.972...","{'class': 'neutral', 'polarity': 0.0, 'subject...","{ ""class"": ""neutral"", ""score"": 0.972, ..."
3100,3452,2023-10-13 15:55:00,"{'class': 'neutral', 'polarity': 0.0, 'subject...",CryptoNews,blockchain,The AutoritÃ© de ContrÃ´le Prudentiel et de RÃ...,French Regulator Concludes DeFi Public Consult...,https://cryptonews.com/news/french-regulator-c...,"{\n ""class"": ""neutral"",\n ""score"": 0.996...","{'class': 'neutral', 'polarity': 0.0, 'subject...","{ ""class"": ""neutral"", ""score"": 0.996, ..."
3101,12399,2023-04-06 21:31:59,"{'class': 'neutral', 'polarity': 0.0, 'subject...",CoinTelegraph,bitcoin,Researchers looking at the “crypto carry” rate...,Smaller investors can have outsized impact on ...,https://cointelegraph.com/news/smaller-investo...,"{\n ""class"": ""neutral"",\n ""score"": 0.925...","{'class': 'neutral', 'polarity': 0.0, 'subject...","{ ""class"": ""neutral"", ""score"": 0.925, ..."
3102,4868,2023-09-15 16:11:00,"{'class': 'neutral', 'polarity': 0.0, 'subject...",CryptoNews,blockchain,Digital asset miners Core Scientific and Celsi...,Celsius and Core Scientific Reach $45 Million ...,https://cryptonews.com/news/celsius-core-scien...,"{\n ""class"": ""positive"",\n ""score"": 0.87...","{'class': 'neutral', 'polarity': 0.0, 'subject...","{ ""class"": ""positive"", ""score"": 0.875, ..."


In [67]:
import pandas as pd
import ast  # To safely evaluate string representations of dictionaries
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error, confusion_matrix, classification_report

# Example DataFrame structure
# df = pd.DataFrame({
#     'date': ['2023-12-19 06:40:41', ...],
#     'sentiment': ["{'class': 'negative', 'polarity': -0.1, 'subjectivity': 0.6}", ...],
#     'Sentiment': ["{'class': 'neutral', 'score': 0.9999802112579346, 'polarity': -0.1, 'subjectivity': 0.6}", ...],
# })

df = temp

# Convert string representations of dictionaries into actual dictionaries
df['true_sentiment'] = df['sentiment'].apply(ast.literal_eval)
df['predicted_sentiment'] = df['Sentiment_LLM_Json'].apply(ast.literal_eval)

# Extract individual components for comparison
df['true_class'] = df['true_sentiment'].apply(lambda x: x['class'])
df['predicted_class'] = df['predicted_sentiment'].apply(lambda x: x['class'])
df['true_polarity'] = df['true_sentiment'].apply(lambda x: x['polarity'])
df['predicted_polarity'] = df['predicted_sentiment'].apply(lambda x: x['polarity'])
df['true_subjectivity'] = df['true_sentiment'].apply(lambda x: x['subjectivity'])
df['predicted_subjectivity'] = df['predicted_sentiment'].apply(lambda x: x['subjectivity'])

# 1. Classification accuracy for 'class'
accuracy = accuracy_score(df['true_class'], df['predicted_class'])
print(f"Classification Accuracy: {accuracy:.2f}")

# 2. Confusion Matrix and Classification Report for 'class'
conf_matrix = confusion_matrix(df['true_class'], df['predicted_class'], labels=['negative', 'neutral', 'positive'])
print("Confusion Matrix:\n", conf_matrix)
print("\nClassification Report:\n", classification_report(df['true_class'], df['predicted_class']))

# 3. Mean Squared Error and Mean Absolute Error for 'polarity' and 'subjectivity'
polarity_mse = mean_squared_error(df['true_polarity'], df['predicted_polarity'])
polarity_mae = mean_absolute_error(df['true_polarity'], df['predicted_polarity'])
subjectivity_mse = mean_squared_error(df['true_subjectivity'], df['predicted_subjectivity'])
subjectivity_mae = mean_absolute_error(df['true_subjectivity'], df['predicted_subjectivity'])

print(f"Polarity MSE: {polarity_mse:.4f}, Polarity MAE: {polarity_mae:.4f}")
print(f"Subjectivity MSE: {subjectivity_mse:.4f}, Subjectivity MAE: {subjectivity_mae:.4f}")


SyntaxError: invalid syntax (<unknown>, line 0)

In [56]:
import pandas as pd
import ast
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error, confusion_matrix, classification_report

# Clean the string representation to remove unwanted escape characters
def clean_string(input_string):
    # Remove backslashes and fix double quotes if necessary
    return input_string.replace('\\"', '"').replace("\\'", "'")

# Example DataFrame (replace with actual data)
df = temp

# Apply the cleaning function before evaluating the string representation of the dictionaries
df['true_sentiment'] = df['sentiment'].apply(lambda x: ast.literal_eval(clean_string(x)))
df['predicted_sentiment'] = df['Sentiment_LLM_Json'].apply(lambda x: ast.literal_eval(clean_string(x)))

# Extract individual components for comparison
df['true_class'] = df['true_sentiment'].apply(lambda x: x['class'])
df['predicted_class'] = df['predicted_sentiment'].apply(lambda x: x['class'])
df['true_polarity'] = df['true_sentiment'].apply(lambda x: x['polarity'])
df['predicted_polarity'] = df['predicted_sentiment'].apply(lambda x: x['polarity'])
df['true_subjectivity'] = df['true_sentiment'].apply(lambda x: x['subjectivity'])
df['predicted_subjectivity'] = df['predicted_sentiment'].apply(lambda x: x['subjectivity'])

# 1. Classification accuracy for 'class'
accuracy = accuracy_score(df['true_class'], df['predicted_class'])
print(f"Classification Accuracy: {accuracy:.2f}")

# 2. Confusion Matrix and Classification Report for 'class'
conf_matrix = confusion_matrix(df['true_class'], df['predicted_class'], labels=['negative', 'neutral', 'positive'])
print("Confusion Matrix:\n", conf_matrix)
print("\nClassification Report:\n", classification_report(df['true_class'], df['predicted_class']))

# 3. Mean Squared Error and Mean Absolute Error for 'polarity' and 'subjectivity'
polarity_mse = mean_squared_error(df['true_polarity'], df['predicted_polarity'])
polarity_mae = mean_absolute_error(df['true_polarity'], df['predicted_polarity'])
subjectivity_mse = mean_squared_error(df['true_subjectivity'], df['predicted_subjectivity'])
subjectivity_mae = mean_absolute_error(df['true_subjectivity'], df['predicted_subjectivity'])

print(f"Polarity MSE: {polarity_mse:.4f}, Polarity MAE: {polarity_mae:.4f}")
print(f"Subjectivity MSE: {subjectivity_mse:.4f}, Subjectivity MAE: {subjectivity_mae:.4f}")


SyntaxError: invalid syntax (<unknown>, line 0)

In [72]:
import json
from sklearn.metrics import accuracy_score

def get(x):
    try:
        return json.loads(x)['class']
    except:
        return 'neutral'
    
# Extract 'class' from 'sentiment' column
df['true_class'] = df['sentiment'].apply(get)

# Define 'get' function to handle possible errors when parsing JSON

# Extract 'class' from 'Sentiment_LLM_Json' column using the get function
df['predicted_class'] = df['Sentiment_LLM_Json'].apply(get)

# Calculate accuracy between true_class and predicted_class
accuracy = accuracy_score(df['true_class'], df['predicted_class'])
print(f"Accuracy between sentiment and Sentiment_LLM_Json: {accuracy:.2f}")


Accuracy between sentiment and Sentiment_LLM_Json: 0.38


In [70]:
df['Sentiment_LLM_Json'][10]

'{    "class": "neutral",    "score": 0.920,    "polarity": 0.1,    "subjectivity": 0.4}'